# Notebook: Final pipelines

#### The dataset analyzed includes data from SteamDB and Game_Data.
#### The analyses are based on the best models from analyses_nlp_comparison and analyses_nlp_merged_data_1.
#### Additional grid searches are conducted to find the best hyperparameters to use in the final pipelines.

#### Step 1: Imports and functions
#### Step 2: NLP basics
#### Step 3: Fit final models and save for use in aim predictions
#### Step 4: Function for prediction of aim data


## Step 1: Imports, functions and classes

In [1]:

#########################################
## Requirements:
## Users might need to manually download stopwords:
nltk.download('stopwords')
## Additionally, en_core_web_sm has to be downloaded manually:
# in terminal: python -m spacy download en_core_web_sm
#########################################

## General imports
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import matplotlib.pyplot as plt
from warnings import simplefilter
from collections import Counter
import pickle 
import sys
import gzip

## Imports for NLP
import nltk, re, spacy, string
from spacy.lang.en.examples import sentences
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

## Imports for analyses
import statsmodels.formula.api as smf
import statsmodels.api as sm
import statsmodels
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge, LinearRegression, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

## Imports for UI
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State

## Imports from analyses_tools (local)
sys.path.append('../../')
from analyses_tools import oh_encoder, NLPAnalyzer
from utilities import URLMerge

## Function to filter entries in detected_technologies for engines used

def filter_engine_entries(text):
    """Extract engine information from detected_technologies"""
    # only keep "Engine" entries for detected_technologies:
    if isinstance(text, str):
        entries = text.replace('; ', ', ').replace(" ", "").split(',')
        filtered_entries = [entry.replace("Engine.", "") for entry in entries if entry.startswith('Engine.')]
        cleaned_text = '; '.join(filtered_entries)    
        if cleaned_text == "":
            return "Unknown"
        else:
            return cleaned_text
    else:
        return "Unknown"


## Function for counting genres

def count_genres(genres):
    """Count number of genres."""
    if pd.isna(genres) or genres == '':
        return 0
    return len(genres.split(','))


## Function for extracting word count and average length

def calculate_description_metrics(description):
    """Split the description into words and extract word count and average length."""
    if pd.isna(description):
        return 0, 0.0 
    ## Split description
    words = re.findall(r'\b\w+\b', description)
    # Calculate word count
    word_count = len(words)
    # Calculate average word length
    if word_count > 0:
        avg_word_length = sum(len(word) for word in words) / word_count
    else:
        avg_word_length = 0
    return word_count, avg_word_length


## Function for data preparation

def data_preparation(df):
    """Take a dataframe, prepare it for use in NLP and analyses and return prepared dataframe.
    
    Args:
        df (Dataframe): Original dataframe to be prepared
        
    Returns:
        df (Dataframe): Prepared dataframe
        
    """
   
    def count_entries(text):
        """Helper function to count entries in lists"""
        entries = text.split('; ')
        return Counter(entries)
    def replace_entries(text, other_entries):
        """Helper function to replace entries in lists"""
        entries = text.split('; ')
        replaced_entries = list(set(['Misc' if entry in other_entries else entry for entry in entries]))
        return '; '.join(replaced_entries)
    
    df = df.copy()
    
    ###############################    
    ## Data Cleaning & Recoding
    ###############################
    
    ## drop columns not used in analyses
    df.drop(['sid', 'store_promo_url', 'published_meta', 'published_stsp', 'published_hltb',
             'published_igdb', 'image', 'current_price', 'discount', 'publisher', 'developer',
             'gfq_url', 'gfq_difficulty_comment', 'gfq_rating_comment', 'gfq_length_comment',
             'hltb_url', 'meta_url', 'igdb_url', 'Unnamed: 0', 'game', 'steam_url', 'release', 
             'positive_reviews', 'negative_reviews', "review_percentage", 'primary_genre', 'store_genres', 
             'store_asset_mod_time', 
             'players_right_now', '24_hour_peak', 'all_time_peak', 'all_time_peak_date'], 
            axis=1, inplace=True)

    ## publish date as timedelta
    df["published_store"] = pd.to_datetime(df["published_store"]) - pd.Timestamp(1997, 1, 1)
    df["published_store"] = df["published_store"].apply(lambda value: value.days)
     
    ## missing data 1: If language or voiceover is missing, set to "One_unknown"
    df.loc[df["languages"].isna(), "languages"] = "Unknown"
    df.loc[df["voiceovers"].isna(), "voiceovers"] = "Unknown"

    ## delete games without English as language:
    count_no_en = 0
    for x in df.index:
        if "english" not in df.loc[x,"languages"].lower():
            count_no_en += 1
            df = df.drop(labels=x, axis=0)
    print(f"Games without English language: {count_no_en}")
             
    ## use only number of languages and voiceovers
    df["languages"] = df["languages"].apply(lambda value: len(value.split(",")))
    df["voiceovers"] = df["voiceovers"].apply(lambda value: len(value.split(",")))
      
    ## missing data 2: drop columns with more than 75% missing data:
    for col in df.columns:
        if df[col].isna().sum() > df.shape[0]*0.75:
            df.drop(col, axis=1, inplace=True) 
    
    ## One-Hot-Encoding

    ## Filter entries in detected_technologies
    df['engine'] = df['detected_technologies'].apply(filter_engine_entries)
    df = df.drop('detected_technologies', axis=1)

    ## Extract word count and average length from description
    df['description_count'], df['description_length'] = zip(*df['description'].apply(calculate_description_metrics))
    
    ## Count number of genres
    df['genres_count'] = df['genres'].apply(count_genres)
       
    ## Splitting mutliple entries
    ## split strings in genre and platform columns
    df['genres'] = df['genres'].apply(lambda x: x.split(','))
    df['platforms'] = df['platforms'].apply(lambda x: x.split(','))
    
    # Count occurences of entries in "engine"
    entry_counts = df['engine'].apply(count_entries).sum()
    # Identify entries with less than 10 occurences
    other_entries = {entry for entry, count in entry_counts.items() if count < 50}
    # Replace entries
    df['engine'] = df['engine'].apply(lambda lst: replace_entries(lst, other_entries))
    df['engine'] = df['engine'].apply(lambda x: x.replace('; ', ', ').replace(" ", "").split(','))
    
    ## replace genres
    df['genres'] = df['genres'].apply(lambda genres: list(set(['Indie' if genre == 'Инди' else genre for genre in genres])))
    df['genres'] = df['genres'].apply(lambda genres: list(set(['Adventure' if genre == 'Приключенческие игры' else genre for genre in genres])))
    
    ## One-Hot Encoding
    df["genres"] = df["genres"].fillna("Unknown")
    df["platforms"] = df["platforms"].fillna("Unknown")
    df["engine"] = df["engine"].fillna("Unknown")
    df = oh_encoder(df, "genres")
    df = oh_encoder(df, "platforms")
    df = oh_encoder(df, "engine")
   
    ## Generate indicators for multi-player games from categories:
    df['Multiplayer'] = df['categories'].apply(
        lambda x: 1 if x and ('Multi-player' in x or 'Massively_Multiplayer' in x) else 0
        )

    ## treat some extreme outliers
    train_df.loc[train_df["hltb_single"]>100, "hltb_single"] = 100
    train_df.loc[train_df["full_price"]>20000, "full_price"] = 20000

    df = df.drop('categories', axis=1)
    
    return df


def data_preparation_aim(train_df, df, is_datetime=False):
    df=df.copy()

    list_of_engines = ["Source", "Unknown", "MonoGame", "AdventureGameStudio", 
                "Unity", "CryEngine", "Solar2D", "KiriKiri", 
                "XNA", "FNA", "Unreal", "Godot", 
                "Construct", "Cocos", "Adobe_AIR", "TyranoBuilder", 
                "Torque", "GameGuru", "RenPy", "OGRE", 
                "RPGMaker", "Love2D", "GameMaker", "Lime_OR_OpenFL",
                "BlenderGameEngine"]
                
    
    ## publish date as timedelta
    if is_datetime==True:
        df["published_store"] = df["published_store"] - pd.Timestamp(1997, 1, 1)
    else:   
        df["published_store"] = pd.to_datetime(df["published_store"]) - pd.Timestamp(1997, 1, 1)
    df["published_store"] = df["published_store"].apply(lambda value: value.days)

    ## process engines
    game_engines=[]
    for entry in df.loc[0,"engine"]:
        if entry not in list_of_engines:
            game_engines.append("Misc")
        else:
            game_engines.append(entry)
    df.loc[0,"engine"] = list(set(game_engines))

    ## Extract word count and average length from description
    df['description_count'], df['description_length'] = zip(*df['description'].apply(calculate_description_metrics))
    
    ## Count number of genres
    df['genres_count'] = df['genres'].apply(count_genres)
       
    ## Splitting mutliple entries
    ## split strings in genre and platform columns
    df['genres'] = df['genres'].apply(lambda x: x.split(','))
    df['platforms'] = df['platforms'].apply(lambda x: x.split(','))
    
    ## One-Hot Encoding
    df["genres"] = df["genres"].fillna("Unknown")
    df["platforms"] = df["platforms"].fillna("Unknown")
    df["engine"] = df["engine"].fillna("Unknown")
    df = oh_encoder(df, "genres")
    df = oh_encoder(df, "platforms")
    df = oh_encoder(df, "engine")
    
    # Ensure same columns in aim_df as in train_df
    for col in train_df.columns:
        if col not in df.columns:
            df[col] = 0
    df = df[train_df.columns]

    ## apply text cleaner
    df["description_clean_nonum"] = df["description"].apply(text_cleaner)

    return df


def text_cleaner(sentence):
    """Take a string, clean it for use in vectorization and return cleaned string.
    
    Args:
        sentence (string): Original string to be cleaned
        
    Returns:
        doc_str (string): Cleaned String
        
    """
    
    ## counter
    global call_count 
    call_count += 1
    if call_count%1000 == 0:
        print(call_count)
    if sentence is None:
        doc_str = ""
    else:
        ## OPTIONAL: delete html tags (tags can be excluded if one wants to limit analyses to ignore this information):
        # sentence = re.sub("<.*?>", "", sentence)
        
        ## tokenize and delete pronouns, stopwords and punctuation
        doc = nlp(sentence)
        clean_doc = [token.lemma_.lower() for token in doc if (token.pos_ !="PRON") and (token.lemma_ not in stopWords) and (token.lemma_ not in punctuations)]
        ## rejoin texts
        doc_str = " ".join(clean_doc)
        ## deleting points, tabs, spaces and line breaks
        doc_str = re.sub("[\s]+", " ", doc_str)
        ## deleting numbers
        doc_str = re.sub(r'\d+', '', doc_str) 
    return doc_str


## Adjusting display
pd.set_option('display.max_rows', 200) # display more rows
pd.set_option('display.max_columns', 50) # display more columns
pd.set_option('display.float_format', '{:.2f}'.format) # display numbers as decimals

## Suppress some warnings 
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

## Step 2: Data preparation and NLP basics

In [2]:

## skips NLP for train data if pickel with it was saved already to drastically reduce time needed

## load language model
nlp = spacy.load("en_core_web_sm")

## Clean "description"
stopWords = stopwords.words("english")
punctuations = string.punctuation


try:
    train_df = pd.read_pickle("../../data/train_df_nlp.pkl", compression='bz2')
    
except:

    ## load and prepare data
    train_df = pd.read_pickle("../../data/df_merge1.pkl", compression='bz2')
    train_df = data_preparation(train_df)
    display(train_df.dtypes)
    
    ## Applying text_cleaner to description
    print("*"*50 + "\nStarting text cleaner\n" + "*"*50) 
    ## Adding a global counter to print in text_cleaner function since it takes a long time
    call_count = 0
    ## Using text_cleaner
    train_df["description_clean_nonum"] = train_df["description"].apply(text_cleaner)
    
    train_df.to_pickle("../../data/train_df_nlp.pkl", compression='bz2')



## Step 3: Fit final models and save for use in aim predictions

In [3]:

## load vocabulary
with open("../../data/extracted_word_index.pkl", "rb") as handle:
    word_index = pickle.load(handle)
custom_vocabulary = word_index[1]

## load merged data 
df = pd.read_pickle("../../data/train_df_nlp.pkl", compression='bz2')


## Pipelines

def final_fitter(df, target_var, custom_vocabulary, poly_degree=2, rf_min_samples_leaf=3):
    
    df = df.copy()

    ## delete missings
    df = df.dropna(axis=0, how="any")

    ## Reset index
    df.reset_index()

    features = df.drop(target_var, axis=1)
    target = df[target_var]
    
    # Define the columns_to_scale and other_columns lists using a list comprehension
    columns_to_scale = [
        col for col in features.columns
        if features[col].nunique() > 2 and pd.api.types.is_numeric_dtype(features[col])
    ]
    
    other_columns = [
        col for col in features.columns
        if col not in columns_to_scale and col != 'description_clean_nonum'
    ]

    # Pipeline for numeric features
    pipeline_num = Pipeline([
        ('scaling', StandardScaler()),
        ('poly', PolynomialFeatures(degree=poly_degree, include_bias=False))
    ])
    
    # Pipeline for text processing
    text_transformer = Pipeline([
        ('vectorizer', TfidfVectorizer(vocabulary=custom_vocabulary, stop_words='english'))
    ])

    # Preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', pipeline_num, columns_to_scale),
            ('text', text_transformer, 'description_clean_nonum'),
            ('other', 'passthrough', other_columns)
        ])
    
    # Main Pipeline with RandomForestRegressor
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(n_estimators=100, min_samples_leaf=rf_min_samples_leaf, n_jobs=-1))
    ])

    return pipeline.fit(features, target)




In [4]:

## fitting final models with best hyperparameters


final_model_owners = final_fitter(df.drop(["name", "developers", "publishers",
                                      "tags", "achievements", "gfq_rating", "description", "voiceovers",
                                      "peak_players", "total_reviews", "rating", "store_uscore", 
                                      "igdb_popularity"], axis=1), 'stsp_owners', custom_vocabulary, 1, 15)



final_model_rating = final_fitter(df.drop(["name", "developers", "publishers",
                                      "tags", "achievements", "gfq_rating", "description", "voiceovers",
                                      "peak_players", "total_reviews", "store_uscore", "stsp_owners", 
                                      "igdb_popularity"], axis=1), 'rating', custom_vocabulary, 2, 3)



final_model_uscore = final_fitter(df.drop(["name", "developers", "publishers",
                                      "tags", "achievements", "gfq_rating", "description", "voiceovers",
                                      "peak_players", "total_reviews", "rating", "stsp_owners", 
                                      "igdb_popularity"], axis=1), 'store_uscore', custom_vocabulary, 2, 3)

models = [final_model_owners, final_model_rating, final_model_uscore]


## save models as pickle
with gzip.open("../../models/final_models.pkl.gz", "wb") as f:
    pickle.dump(models, f)


## Step 4: Function for prediction of aim data

In [5]:
####################################################################
## optional: load models from pickle object
# with gzip.open("../../models/final_models.pkl.gz", "rb") as f:
#     models = pickle.load(f)
####################################################################

## Function for Predictions: 

def predictor(models, df_aim):

    ## predictions
    y_owners = models[0].predict(df_aim)
    y_rating = models[1].predict(df_aim)
    y_uscore = models[2].predict(df_aim)

    return y_owners, y_rating, y_uscore


In [6]:

## Testing predictions with examplary data

## example data 1
data_1 = {'published_store': ['2024-08-01'], 
          'name': ["New Game 1"],     
          'description': ['An action game with different enemies you need to play.'],
          'full_price': [199],
          'languages': [1],
          'hltb_single': [2.0],        
          'genres': ['Casual'],
          'platforms': ['WIN'],
          'engine': ['Unity'],
          'Multiplayer': [0],
          }

df_aim = pd.DataFrame.from_dict(data_1)
call_count = 0
df_aim = data_preparation_aim(df, df_aim)

## make predictions
y_owners, y_rating, y_uscore = predictor(models, df_aim)

for prediction in [y_owners, y_rating, y_uscore]:
    print(prediction)

## example data 2
data_2 = {'published_store': ['2025-08-01'], 
          'name': ["New Game 2"],     
          'description': ['''An action game with great graphics and battles. 
                          You can... 
                          <ul>
                          <li>explore the new world with many new areas,</li>
                          <li>find new friends,</li>
                          <li>gain new unique powers,</li>
                          <li>get strong,</li>
                          <li>solve new unique puzzles,</li> 
                          <li>overcome challenges,</li>
                          <li>enjoy adventures,</li>
                          <li>live a live,</li>
                          <li>find ways to complete the story,</li>
                          <li>make things,</li>
                          <li>and become a strong and friendly character.</li>
                          </ul)
                          '''],
          'full_price': [4999],
          'languages': [12],
          'hltb_single': [20.0],        
          'genres': ['Action'],
          'platforms': ['WIN, LNX, MAC'],
          'engine': ['Unreal'],
          'Multiplayer': [1],
          }

df_aim = pd.DataFrame.from_dict(data_2)
call_count = 0
df_aim = data_preparation_aim(df, df_aim)

## make predictions
y_owners, y_rating, y_uscore = predictor(models, df_aim)

for prediction in [y_owners, y_rating, y_uscore]:
    print(prediction)

[259198.95163818]
[63.74261814]
[60.79136183]
[2423534.8192595]
[74.57429452]
[75.12423232]


In [7]:
## add prediction for sales

## function for predicting sales

def predictor_sales(model_ols, df_aim_sales):

    df_aim_sales = df_aim_sales[["stsp_owners"]]
    ## df_aim_sales = df_aim_sales[["const", "stsp_owners"]]
    
    ## prediction
    y_sales = model_ols.predict(df_aim_sales)

    if y_sales[0] < 0:
        return [0]
    else:
        return y_sales*1000000

## fit statistical model for sales based on rating, uscore and owners

df_steamdb = pd.read_json(r'../../data/steamdb.json')
df_game_data = pd.read_csv(r'../../data/game_data_all.csv')
df_sales = pd.read_csv(r'../../data/vgchartz-2024.csv')

from utilities import URLMerge

df_merge_1 = URLMerge(df_steamdb, 'store_url', df_game_data, 'link')

df_merge_1["title"] = df_merge_1["name"]

df_merge_2 = pd.merge(df_sales, df_merge_1, on="title", how="inner")

df_merge_2 = df_merge_2[["total_sales", "stsp_owners"]]
df_merge_2.dropna(how="any", inplace=True)

display(df_merge_2.describe())

model_ols = sm.OLS(df_merge_2["total_sales"], df_merge_2.drop("total_sales", axis=1)).fit()
## model_ols = sm.OLS(df_merge_2["total_sales"], sm.add_constant(df_merge_2.drop("total_sales", axis=1))).fit()
adj_r2_sm = model_ols.rsquared_adj
display(model_ols.summary())
print("*"*50, "\n", "R-squared for Statsmodels OLS (train data):", model_ols.rsquared)
print("*"*50, "\n", "Adjusted R-squared for Statsmodels OLS (train data):", adj_r2_sm)


## save model as pickle
with gzip.open("../../models/final_model_sales.pkl.gz", "wb") as f:
    pickle.dump(model_ols, f)


,total_sales,stsp_owners
count,1443.00,1443.00
mean,0.37,782176.02
std,1.28,2579473.29
min,0.00,10000.00
25%,0.02,35000.00
50%,0.06,150000.00
75%,0.23,750000.00
max,20.32,35000000.00


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:            total_sales   R-squared (uncentered):                   0.346
Model:                            OLS   Adj. R-squared (uncentered):              0.345
Method:                 Least Squares   F-statistic:                              762.3
Date:                Tue, 18 Jun 2024   Prob (F-statistic):                   4.69e-135
Time:                        00:01:32   Log-Likelihood:                         -2158.2
No. Observations:                1443   AIC:                                      4318.
Df Residuals:                    1442   BIC:                                      4324.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
stsp_owners  2.913e-07   1.06e-08     27.609      0.000    2.71e-07    3.12e-07
==============================================================================
Omnibus:                     1648.013   Durbin-Watson:                   1.169
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           355577.245
Skew:                           5.367   Prob(JB):                         0.00
Kurtosis:                      79.149   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

************************************************** 
 R-squared for Statsmodels OLS (train data): 0.34581754752274263
************************************************** 
 Adjusted R-squared for Statsmodels OLS (train data): 0.3453638842408583
